# Module

In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
from tqdm import tqdm_notebook as tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")
gc.enable()

In [2]:
pd.set_option('max_rows', 500)
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)

# Load Data

In [3]:
train_raw = pd.read_csv('./data/train.csv')
test_raw = pd.read_csv('./data/test.csv')
train_raw.shape, test_raw.shape

((200000, 202), (200000, 201))

In [5]:
train = train_raw.copy()
test = test_raw.copy()

In [6]:
col_list = train.columns[2:]

In [7]:
train_0 = train[train.target == 0]
train_1 = train[train.target == 1]

In [8]:
pb_idx = np.load('./data_temp/public_LB.npy')
pv_idx = np.load('./data_temp/private_LB.npy')

In [9]:
test_pb = test.iloc[pb_idx].sort_index().copy()
test_pv = test.iloc[pv_idx].sort_index().copy()

test_real = test_pb.append(test_pv)

In [10]:
data = train.append(test_real)

In [11]:
oof = pd.read_csv('./data_temp/new_bbiggu.csv')

In [12]:
bbiggu_0 = oof[(oof.target == 0) & (oof.pred > 0.1)]
bbiggu_1 = oof[(oof.target == 1) & (oof.pred < 0.1)]
normal_0 = oof[(oof.target == 0) & (oof.pred < 0.1)]
normal_1 = oof[(oof.target == 1) & (oof.pred > 0.1)]

In [13]:
bbiggu_0 = train[train.ID_code.isin(bbiggu_0.ID_code)]
bbiggu_1 = train[train.ID_code.isin(bbiggu_1.ID_code)]
normal_0 = train[train.ID_code.isin(normal_0.ID_code)]
normal_1 = train[train.ID_code.isin(normal_1.ID_code)]

In [80]:
bbiggu_0_id_code = bbiggu_0.ID_code
bbiggu_1_id_code = bbiggu_1.ID_code
normal_0_id_code = normal_0.ID_code
normal_1_id_code = normal_1.ID_code

# 삐꾸학습기

In [114]:
bbiggu_1 = data[data.ID_code.isin(bbiggu_1_id_code)]
normal_0 = data[data.ID_code.isin(normal_0_id_code)]

In [118]:
bbiggu_1.target = 1
normal_0.target = 0

In [119]:
train = bbiggu_1.append(normal_0)

In [122]:
train.sort_index(inplace=True)

# Aug

In [96]:
def aug(data, col_list):
    b_0 = data[data.ID_code.isin(bbiggu_0_id_code)]
    n_0 = data[data.ID_code.isin(normal_0_id_code)]
    b_1 = data[data.ID_code.isin(bbiggu_1_id_code)]
    n_1 = data[data.ID_code.isin(normal_1_id_code)]
    
    for col in tqdm(col_list):
        temp_0 = b_0[col].values.tolist() + n_0[col].values.tolist()
        temp_1 = b_1[col].values.tolist() + n_1[col].values.tolist()
        
        np.random.shuffle(temp_0)
        np.random.shuffle(temp_1)
        
        b_0[col] = temp_0[:len(b_0)]
        b_1[col] = temp_1[:len(b_1)]
    
    return data.append(b_0[col_list]).append(b_1[col_list])

In [52]:
unique_df = data[['ID_code']]
con_df = data[['ID_code']]

In [53]:
for col in tqdm(col_list):
    unique_df[col] = data[col].map(((data[col].value_counts() == 1) * 1).to_dict())
    con_df[col] = data[col].map((~(data[col].value_counts() == 1) * 1).to_dict())

In [54]:
for col in tqdm(col_list):
    data[col + '_unique'] = np.around(data[col] * unique_df[col], 4)
    data[col + '_con'] = np.around(data[col] * con_df[col], 4)

In [55]:
for col in tqdm(col_list):
    data[col + '_value_counts'] = data[col].map(data[col].value_counts().to_dict())

In [134]:
train = data[~data.target.isna()]
test = data[data.target.isna()]

In [136]:
target = train['target']

In [137]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average': False,
    'boost': 'gbdt',
    'feature_fraction_seed': 47,
    'feature_fraction': 0.041,
    'learning_rate': 0.01,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1,
    'num_threads': 8
}

In [138]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):    
    print("fold n°{}".format(fold_))
    
    trn_data = train.iloc[trn_idx]
    val_data = train.iloc[val_idx]
    
#     print("augmentation ...")
#     trn_data = aug(trn_data, [c for c in train.columns if c not in ['ID_code', 'target'] + col_list.tolist()])
    
    trn_data = lgb.Dataset(trn_data[train_columns], label=trn_data.target)
    val_data = lgb.Dataset(val_data[train_columns], label=val_data.target)

    num_round = 50000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=400, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_lgb[val_idx])))
            
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[400]	training's auc: 0.907157	valid_1's auc: 0.897085
[800]	training's auc: 0.913871	valid_1's auc: 0.901772
[1200]	training's auc: 0.919338	valid_1's auc: 0.905461
[1600]	training's auc: 0.923635	valid_1's auc: 0.907966
[2000]	training's auc: 0.927277	valid_1's auc: 0.909854
[2400]	training's auc: 0.930513	valid_1's auc: 0.911144
[2800]	training's auc: 0.933348	valid_1's auc: 0.912134
[3200]	training's auc: 0.935778	valid_1's auc: 0.912831
[3600]	training's auc: 0.937919	valid_1's auc: 0.913335
[4000]	training's auc: 0.939835	valid_1's auc: 0.913717
[4400]	training's auc: 0.941678	valid_1's auc: 0.913916
[4800]	training's auc: 0.943566	valid_1's auc: 0.91402
[5200]	training's auc: 0.945367	valid_1's auc: 0.914287
[5600]	training's auc: 0.947104	valid_1's auc: 0.914424
[6000]	training's auc: 0.948882	valid_1's auc: 0.914605


KeyboardInterrupt: 

In [130]:
data['bbiggu_pred'] = np.nan

In [131]:
data.loc[data.ID_code.isin(train.ID_code), 'bbiggu_pred'] = oof_lgb

In [129]:
data.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_11,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_12,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_13,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_14,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_15,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_16,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_17,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_18,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_19,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_2,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_3,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_4,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_5,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_6,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_7,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_8,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_9,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_0_unique,var_0_con,var_1_unique,var_1_con,var_2_unique,var_2_con,var_3_unique,var_3_con,var_4_unique,var_4_con,var_5_unique,var_5_con,var_6_unique,var_6_con,var_7_unique,var_7_con,var_8_unique,var_8_con,var_9_unique,var_9_con,var_10_unique,var_10_con,var_11_unique,var_11_con,var_12_unique,var_12_con,var_13_unique,var_13_con,var_14_unique,var_14_con,var_15_unique,var_15_con,var_16_unique,var_16_con,var_17_unique,var_17_con,var_18_unique,var_18_con,var_19_unique,var_19_con,var_20_unique,var_20_con,var_21_unique,var_21_con,var_22_unique,var_22_con,var_23_unique,var_23_con,...,var_175_unique,var_175_con,var_176_unique,var_176_con,var_177_unique,var_177_con,var_178_unique,var_178_con,var_179_unique,var_179_con,var_180_unique,var_180_con,var_181_unique,var_181_con,var_182_unique,var_182_con,var_183_unique,var_183_con,var_184_unique,var_184_con,var_185_unique,var_185_con,var_186_unique,var_186_con,var_187_unique,var_187_con,var_188_unique,var_188_con,var_189_unique,var_189_con,var_190_unique,var_190_con,var_191_unique,var_191_con,var_192_unique,var_192_con,var_193_unique,var_193_con,var_194_unique,var_194_con,var_195_unique,var_195_con,var_196_unique,var_196_con,var_197_unique,var_197_con,var_198_unique,var_198_con,var_199_unique,var_199_con,var_0_value_counts,var_1_value_counts,var_2_value_counts,var_3_value_counts,var_4_value_counts,var_5_value_counts,var_6_value_counts,var_7_value_counts,var_8_value_counts,var_9_value_counts,var_10_value_counts,var_11_value_counts,var_12_value_counts,var_13_value_counts,var_14_value_counts,var_15_value_counts,var_16_value_counts,var_17_value_counts,var_18_value_counts,var_19_value_counts,var_20_value_counts,var_21_value_counts,var_22_value_counts,var_23_value_counts,var_24_value_counts,var_25_value_counts,var_26_value_counts,var_27_value_counts,var_28_value_counts,var_29_value_counts,var_30_value_counts,var_31_value_counts,var_32_value_counts,var_33_value_counts,var_34_value_counts,var_35_value_counts,var_36_value_counts,var_37_value_counts,var_38_value_counts,var_39_value_counts,var_40_value_counts,var_41_value_counts,var_42_value_counts,var_43_value_counts,var_44_value_counts,var_45_value_counts,var_46_value_counts,var_47_value_counts,var_48_value_counts,var_49_value_counts,var_50_value_counts,var_51_value_counts,var_52_value_counts,var_53_value_counts,var_54_value_counts,var_55_value_counts,var_56_value_counts,var_57_value_counts,var_58_value_counts,var_59_value_counts,var_60_value_counts,var_61_value_counts,var_6